# Pivot and Validate Precinct Level Election Results

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

### Import the datasets

In [2]:
statewide_election_results_file_path = '20181106__ny__general__precinct.csv'
statewide_results_df = pd.read_csv(statewide_election_results_file_path,low_memory=False)
statewide_results_df.head(2)

,county,precinct,office,district,candidate,party,votes,election_day,absentee,machine_votes,absentee_hc,affidavit
0,Chemung,Elmira 0901,Governor,NaN,Andrew Cuomo,Dem,139,NaN,NaN,NaN,NaN,NaN
1,Chemung,Elmira 0902,Governor,NaN,Andrew Cuomo,Dem,104,NaN,NaN,NaN,NaN,NaN


In [3]:
official_county_level_election_results_file_path = 'official_county_level_results_2018Governor.csv'
official_county_level_election_results_df = pd.read_csv(official_county_level_election_results_file_path,low_memory=False)
official_county_level_election_results_df = official_county_level_election_results_df.set_index('County')
official_county_level_election_results_df.head(2)

,Andrew M. Cuomo and Kathy C. Hochul (DEM),Marc Molinaro and Julie Killian (REP),Marc Molinaro and Julie Killian (CON),Howie Hawkins and Jia Lee (GRE),Andrew M. Cuomo and Kathy C. Hochul (WOR),Andrew M. Cuomo and Kathy C. Hochul (IND),Andrew M. Cuomo and Kathy C. Hochul (WEP),Marc Molinaro and Julie Killian (REF),Larry Sharpe and Andrew C. Hollister (LBT),Stephanie A. Miner and Michael J. Volpe (SAM),Blank,Void,Scattering,Total Votes by County
County,,,,,,,,,,,,,,
Albany County,54909,38357,6725,3543,2489,1408,886,835,1867,3007,2807,268,195,"117,296"
Allegany County,3254,9234,833,183,80,106,46,65,963,117,462,3,7,"15,353"


The next cell aliases `statewide_results_df` as `df` and `official_county_level_election_results_df` as `odf` because typing fewer characters allows for faster data exploration.  

In [4]:
df = statewide_results_df.copy()
odf = official_county_level_election_results_df.copy()

### Clean `votes`

In [5]:
def remove_commas(df, col_lst):
    """
    remove commas from the string representation of numbers in the cols in col_lst
    :df: DataFrame object
    :col_lst: list of strings that are each the name of a column :df:
    """
    for col in col_lst:
        if df.dtypes[col] == "object":
            df[col] = df[col].map(lambda s: s.replace(",", ""))
        df = df.astype({col: "float"})
        df = df.astype({col: "int64"})
    return df

df = remove_commas(df[df.votes.notna()], ['votes'])

/Users/baxterdemers/ny2018/venv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### Clean `county`

In [6]:
df.county = df.county.str.strip() + ' County'

### Clean `office`

In [7]:
def clean_office(office):
    if 'gov' in office.lower():
        return 'Governor'
    if office == 'U.S.Senate':
        return 'U.S. Senate'
    if 'U.S.House' in office:
        return 'U.S. House'
    return office.title().strip()

df.office.map(clean_office).unique()

array(['Governor', 'Comptroller', 'Attorney General', 'U.S. Senate',
       'U.S. House', 'State Senate', 'State Assembly'], dtype=object)

In [8]:
df.office = df.office.map(clean_office)

In [9]:
offices = list(df.office.unique())
print(len(offices))
print(offices)

7
['Governor', 'Comptroller', 'Attorney General', 'U.S. Senate', 'U.S. House', 'State Senate', 'State Assembly']


Ensure that all counties data for each office

In [10]:
county_to_offices = df.groupby('county').office.unique()
county_to_missing_offices = county_to_offices.apply(lambda x: set(offices).difference(x))
county_to_missing_offices[county_to_missing_offices != set()]

county
Albany County                                {State Assembly}
Allegany County                                    {Governor}
Broome County                                {State Assembly}
Chautauqua County                            {State Assembly}
Chemung County                               {State Assembly}
Clinton County                               {State Assembly}
Columbia County               {Comptroller, Attorney General}
Cortland County                              {State Assembly}
Erie County                                  {State Assembly}
Genesee County                                     {Governor}
Kings County                                 {State Assembly}
Madison County                               {State Assembly}
Montgomery County                            {State Assembly}
New York County                              {State Assembly}
Niagara County                                  {U.S. Senate}
Oneida County                                {State Assembly}
O

In [11]:
federal_offices = ['Governor', 'U.S. Senate', 'U.S. House']
county_to_missing_federal_offices = county_to_offices.apply(lambda x: set(federal_offices).difference(x))
counties_missing_federal_offices = county_to_missing_federal_offices[county_to_missing_federal_offices != set()]
print(len(counties_missing_federal_offices))
counties_missing_federal_offices

7


county
Allegany County                {Governor}
Genesee County                 {Governor}
Niagara County              {U.S. Senate}
Orange County      {U.S. House, Governor}
Saratoga County              {U.S. House}
Schuyler County                {Governor}
Suffolk County                 {Governor}
Name: office, dtype: object

Seems like Open Elections has more up to date county level files here: https://github.com/openelections/openelections-data-ny/tree/master/2018/counties

* Allegany County                {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__allegany__precinct.csv
* Genesee County                 {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__genesee__precinct.csv
* Niagara County              {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__niagara__precinct.csv
* Orange County      {U.S. House, X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__orange__precinct.csv
* Saratoga County              {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__saratoga__precinct.csv
* Schuyler County                {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__schuyler__precinct.csv
* Suffolk County                 {X} https://github.com/openelections/openelections-data-ny/blob/master/2018/counties/20181106__ny__general__suffolk__precinct.csv 

All set with the expection of missing results for the U.S. House race in Orange County.

### Getting results for U.S. House in Orange County

1. Downloaded `data/parse_results/GE18 -CD 18th.pdf` from https://www.orangecountygov.com/DocumentCenter/View/11663/GE18--CD-18th
2. Used https://pdftables.com/ to convert the pdf to a csv (`data/parse_results/GE18 -CD 18th.csv`)
3. Made some manual tweaks to clean up the handwork of the pdf to csv converter in Numbers (like Excel for Mac OS). Saved it locally at `data/parse_results/GE18-CD-18th-cleaned.csv`
4. Imported below

Since this data is already pivoted, I will incoperate it after I pivot the rest of the data.

In [33]:
df_Orange_HOR = pd.read_csv('data/parse_results/GE18-CD-18th-cleaned.csv')
df_Orange_HOR

,precinct,DEM,REP,CON,WOR,IND,WEP,REF,WRITE IN,BLANK
0,Middletown - Ward 1-1,159,67,7,6,4,2,1,0,11
1,Middletown - Ward 1-2,79,34,3,1,0,0,0,0,3
2,Middletown - Ward 1-3,158,47,7,3,2,1,2,0,15
3,Middletown - Ward 1-4,36,19,0,2,1,0,0,0,3
4,Middletown - Ward 1-5,139,53,7,6,1,0,1,0,9
...,...,...,...,...,...,...,...,...,...,...
329,Woodbury 6,202,191,27,7,6,3,4,0,34
330,Woodbury 7,192,117,13,3,9,2,8,0,15
331,Woodbury 8,175,211,27,3,5,1,2,0,13
332,Woodbury 9,129,36,7,0,1,1,1,0,10


### Patch in Counties that are missing election results

For whatever reason, Open Elections has more complete results at the county level. Counties missing election results for federal offices: 
* Allegany County                {Governor}
* Genesee County                 {Governor}
* Niagara County              {U.S. Senate}
* Orange County      {Governor}
* Saratoga County              {U.S. House}
* Schuyler County                {Governor}
* Suffolk County                 {Governor}

In [12]:
df_Allegany = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__allegany__precinct.csv')
df_Genesee = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__genesee__precinct.csv')
df_Niagara = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__niagara__precinct.csv')
df_Orange = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__orange__precinct.csv')
df_Saratoga = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__saratoga__precinct.csv')
df_Schuyler = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__schuyler__precinct.csv')
df_Suffolk = pd.read_csv('data/county_precinct_election_results_oe/20181106__ny__general__suffolk__precinct.csv')

In [13]:
county_lst = ['Allegany', 'Genesee', 'Niagara', 'Orange', 'Saratoga', 'Schuyler', 'Suffolk']
county_df_lst = [df_Allegany, df_Genesee, df_Niagara, df_Orange, df_Saratoga, df_Schuyler,df_Suffolk]

In [14]:
df = df[~df.county.isin(county_lst)]
df = df.append(county_df_lst)

In [15]:
df[df.county == 'Allegany'].office.unique()

array(['State Assembly', 'State Senate', 'U.S. House', 'U.S. Senate',
       'GOVERNOR/ LT.', 'COMPTROLLER', 'ATTORNEY GENERAL'], dtype=object)

### Repeat cleaning

In [16]:
def clean_office(office):
    if 'gov' in office.lower():
        return 'Governor'
    if office == 'U.S.Senate':
        return 'U.S. Senate'
    if 'U.S.House' in office:
        return 'U.S. House'
    return office.title().strip()

df.office = df.office.map(clean_office)
df = df[df.office.isin(offices)]
df.office.unique()

array(['Governor', 'Comptroller', 'Attorney General', 'U.S. Senate',
       'U.S. House', 'State Senate', 'State Assembly'], dtype=object)

In [17]:
df.loc[df.county.isin(county_lst), 'county'] = df.loc[df.county.isin(county_lst),'county'].str.strip() + ' County'

Ensure that all counties data for each office

In [31]:
county_to_offices = df.groupby('county').office.unique()
county_to_missing_offices = county_to_offices.apply(lambda x: set(offices).difference(x))
county_to_missing_offices = county_to_missing_offices[county_to_missing_offices != set()]
print(len(county_to_missing_offices))
county_to_missing_offices

21


county
Albany County                         {State Assembly}
Broome County                         {State Assembly}
Chautauqua County                     {State Assembly}
Chemung County                        {State Assembly}
Clinton County                        {State Assembly}
Columbia County        {Comptroller, Attorney General}
Cortland County                       {State Assembly}
Erie County                           {State Assembly}
Kings County                          {State Assembly}
Madison County                        {State Assembly}
Montgomery County                     {State Assembly}
New York County                       {State Assembly}
Oneida County                         {State Assembly}
Orange County             {State Assembly, U.S. House}
Queens County                            {Comptroller}
Richmond County                       {State Assembly}
St. Lawrence County                   {State Assembly}
Tompkins County                       {State Assembly}
War

In [30]:
federal_offices = ['Governor', 'U.S. Senate', 'U.S. House']
county_to_missing_federal_offices = county_to_offices.apply(lambda x: set(federal_offices).difference(x))
counties_missing_federal_offices = county_to_missing_federal_offices[county_to_missing_federal_offices != set()]
print(len(counties_missing_federal_offices))
counties_missing_federal_offices

1


county
Orange County    {U.S. House}
Name: office, dtype: object

df_Orange.office.unique()

https://ballotpedia.org/New_York%27s_18th_Congressional_District_election,_2018

### Clean `party`

The oficial parties in New York are [defined as](https://www.elections.ny.gov/CFCommittees.html)

- Democratic
- Republican
- Conservative
- Working Families
- Green
- Libertarian
- Independence
- SAM

Since the official results also included `WEP (Women's Equality Party)` and `REF (Reform Party of New York State)` I will include those as well.

In [23]:
df.party.nunique()

89

In [29]:
len(parties)

10

In [24]:
official_party_to_abbreviation = {
    "Democratic":"DEM",
    "Republican":"REP",
    "Conservative":"CON",
    "Working Families":"WOR",
    "Green":"GRE",
    "Libertarian":"LBT",
    "Independence":"IND",
    "SAM": "SAM",
    "Women's Equality Party":"WEP",
    "Reform Party":"REF",
}

parties = set(official_party_to_abbreviation.values())
misc = {'BLANK, VOID OR SCATTERED', 'WRITEIN', 'OVERVOTE', 'UNDERVOTE', 'UNQUALIFIED WRITE-INS'}

In [25]:
party_lookup_table = {
    'DEMOCRATIC':'DEM',
    '0':'BLANK, VOID OR SCATTERED',
    '1635':'BLANK, VOID OR SCATTERED',
    '4328':'BLANK, VOID OR SCATTERED',
    'BLANK & VOID':'BLANK, VOID OR SCATTERED',
    'BLANKS':'BLANK, VOID OR SCATTERED',
    'BLANKS & VOID':'BLANK, VOID OR SCATTERED',
    'IND': 'IND',
    'REP': 'REP',
 'Write': 'Write',
 'DEM': 'DEM',
 'CON': 'CON',
 'SAM': 'SAM',
     'GRE': 'GRE',
 'WOR': 'WOR',
 'REF': 'REF',
 'WEP': 'WEP',
 'LBT': 'LBT',
 'WRI':'WRITEIN',
 'WRITE IN':'WRITEIN',
 'WRITE INS':'WRITEIN',
 'WRITE-IN':'WRITEIN',
 'WRITE-INS':'WRITEIN',
 'WRITEIN':'WRITEIN',
 'WRITEINS':'WRITEIN',
     'VOID':'BLANK, VOID OR SCATTERED',
 'VOIDS':'BLANK, VOID OR SCATTERED',
 'VOIDS/BLANKS':'BLANK, VOID OR SCATTERED',
    'GREEN':'GRE',
 'GRN':'GRE',
     'BLANK':'BLANK, VOID OR SCATTERED',
 'BLANK VOTES':'BLANK, VOID OR SCATTERED',
 'BLANK/VOID':'BLANK, VOID OR SCATTERED',
     'LIB':'LBT',
 'LIBERTARIAN':'LBT',
    'SCATTER':'BLANK, VOID OR SCATTERED',
 'SCATTERED':'BLANK, VOID OR SCATTERED',
 'SCATTERING':'BLANK, VOID OR SCATTERED',
 'SCATTERINGS':'BLANK, VOID OR SCATTERED',
'CONSERVATIVE':'CON',
         'OVER':'OVERVOTE',
 'OVER VOTES':'OVERVOTE',
 'OVERVOTE':'OVERVOTE',
 'OVERVOTES':'OVERVOTE',
 'UNDER':'UNDERVOTE',
 'UNDER VOTES':'UNDERVOTE',
 'UNDER/OVER':'UNDERVOTE',
 'UNDER/OVER VOTES':'UNDERVOTE',
 'UNDERVOTE':'UNDERVOTE',
 'UNDERVOTES':'UNDERVOTE',
    'REPUBLICAN':'REP',
    'REFORM':'REF',
   'REFORM PARTY':'REF',
    'INDEPENDENCE':'IND',
    'WFP':'WOR',
    'WF':'WOR',
    "WOMEN'S EQUALITY PARTY":'WEP',
    "WOMEN'S EQUALITY":"WEP",
    # ballotpedia lookups
    'GLOGOWSKI':'LBT',
    "CHRISTOPHER B. GARVEY":'LBT',
    'SEAMANS':'DEM',
    'DINAPOLI':'DEM',
    'GILLIBRAND':'DEM',
    'JAMES': 'DEM',
    'COLLINS':'REP',
    'TRICHTER':'REP',
    'CC FARLEY':'REP',
    'WOFFORD':'REP',
    'RANZENHOFER':'REP',
    'HAWLEY':'REP',
    'MCMURRAY':'DEM',
}

def clean_party(party):
    if (type(party) != str):
        return 'nS'
    cleaned_party = party.strip().upper()
    looked_up_party = party_lookup_table.get(cleaned_party, cleaned_party)
    return official_party_to_abbreviation.get(looked_up_party.title(), looked_up_party)

In [26]:
df.loc[df.party.isna(),'party'] = df.candidate
df = df.loc[~(df.party.str.upper().isin([
        'BALLOTS CAST',
        'TOTAL',
        'TOTAL BALLOTS',
        'TOTAL VOTES',
        'REGISTERED VOTERS',
        'FEDERAL',
        'FEDERAL BALLOTS',
        'FEDERAL VOTES',
        'STATE BALLOTS',
        'STATE VOTES',
        'REGISTERED',
        'PUBLIC COUNTER',]))
        ]

ValueError: cannot reindex from a duplicate axis

In [ ]:
df['party'] = df.party.map(clean_party)
df = df[df.party.isin(parties)]
value_counts = df.groupby('party').votes.sum()
print(value_counts[value_counts > 1000].shape)
value_counts[value_counts > 1000].sort_values().tail(60)

In [ ]:
df.office.unique()

In [ ]:
office_to_abbreviation = {
    'Governor':'Gov', 
    'Comptroller':'Compt', 
    'Attorney General':'AG',
    'U.S. Senate':'Sen',
    'U.S. House':'HOR',
    'State Senate':'StSen', 
    'State Assembly':'StAssy'
}

In [ ]:
df.office = df.office.map(office_to_abbreviation)

In [ ]:
df.head()

In [ ]:
piv = pd.pivot_table(df, index = ['county', 'precinct'], columns = ['office','party'], values = ['votes'], aggfunc = np.sum)

In [ ]:
piv.columns = [office + '_' + party for _,office,party in piv.columns]
piv.head()

In [ ]:
piv.groupby('county').sum()

## Validation

Compare the precinct level election results to official county level election results released by New York's Secretary of State

### Clean up the pivoted data

In [ ]:
piv = remove_commas(piv.fillna(0), piv.columns)

In [ ]:
res_gov = piv.groupby('county').sum()[[x for x in piv.columns if x.startswith('Gov')]]
res_gov.head()

In [ ]:
res_gov.columns = [x.split('_')[1] for x in res_gov.columns]
res_gov.head()

In [ ]:
o_gov = odf[[x for x in odf.columns if '(' in x]]
o_gov.columns = [x.split('(')[1][:3] for x in o_gov.columns]
o_gov.head()

In [ ]:
party_lst = list(parties)
party_lst.sort()
party_lst

Order columns the same way in both data sets

In [ ]:
o_gov = o_gov[party_lst]
o_gov.head(2)

In [ ]:
res_gov = res_gov[party_lst]
res_gov.head(2)

Allegany seemingly missing results for the govenors race. Will now check out the `df` for other `county, office` tuples that are missing data.

In [ ]:
cov = df.groupby(['county', 'office']).votes.sum().sort_values()

In [ ]:
cov['Allegany']

In [ ]:
cov

In [ ]:
df.groupby('county').office.unique()